In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [2]:
import jax
import jax.numpy as jnp
from flax import nnx
from functools import partial
from jax_tqdm import scan_tqdm

from algorithms.utils import save_state_multiagent

import numpy as np

# jax.config.update("jax_debug_nans", True)

2025-03-21 11:24:41.969895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742552681.982893  773556 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742552681.986792  773556 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from ernestogym.envs_jax.multi_agent.env import RECEnv

In [4]:
def my_env_creator(params, battery_type, env_type='normal'):
    if env_type == 'normal':
        env = RECEnv(params, battery_type)
    return env

In [5]:
from ernestogym.envs_jax.multi_agent.utils import parameter_generator

In [6]:
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_cheap.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full_cheap.yaml"
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_half_full_cheap.yaml"

ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"


# world = "ernestogym/envs_jax/multi_agent/world_deg_train.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_demands.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_only_trad_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_all_the_same_debug.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_all_the_same_debug_like_single.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_new_clip.yaml"
world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_exp_avg_new_clip.yaml"


# battery_type = 'fading'
# battery_type = 'degrading'
battery_type = 'degrading_dropflow'

params = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    # use_reward_normalization=True

)

params

yaaaas2


{'num_battery_agents': 3,
 'num_passive_houses': 0,
 'batteries': [{'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 288.0,
    'temp_ambient': 298.15,
    'nominal_cost': 1500.0},
   'bounds': {'voltage': {'low': 288.0, 'high': 398.4},
    'current': {'low': -180.0, 'high': 60.0},
    'power': {'low': -71712.0, 'high': 23904.0},
    'temperature': {'low': 273.15, 'high': 323.15},
    'temp_ambient': {'low': 273.15, 'high': 323.15},
    'soc': {'low': 0.2, 'high': 1.0},
    'soh': {'low': 0.6, 'high': 1.0}},
   'init': {'voltage': 310.8,
    'current': 0.0,
    'power': 0.0,
    'temperature': 293.15,
    'temp_ambient': 293.15,
    'soc': 0.5,
    'soh': 1.0}},
  {'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 2

In [7]:
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_cheap.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full_cheap.yaml"
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_half_full_cheap.yaml"

ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"



# world = "ernestogym/envs_jax/multi_agent/world_deg_test.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_demands.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_only_trad_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_all_the_same_debug.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_all_the_same_debug_like_single.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_new_clip.yaml"
world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_exp_avg_new_clip.yaml"


params_testing = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    # use_reward_normalization=True

)

params_testing['step']

yaaaas2


3600

In [8]:
from algorithms.multi_agent_ppo import make_train, train_wrapper
from flax.core.frozen_dict import freeze, unfreeze

In [9]:
num_envs = 4
total_timesteps = 8760 * num_envs * 11

env_type = 'normal'

config = {
    "LR_BATTERIES": 5e-6,
    "LR_REC": 5e-6,
    "LR_BATTERIES_MIN": 5e-8,
    "LR_REC_MIN": 5e-8,
    "NUM_ENVS": num_envs,
    "NUM_STEPS": 8192, #2048 #10000,
    # "TOTAL_TIMESTEPS": 100000 * num_envs,
    "TOTAL_TIMESTEPS": total_timesteps,
    "UPDATE_EPOCHS": 10,
    "NUM_MINIBATCHES": 32, #64,
    "NORMALIZE_ENV": False,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.98,
    "CLIP_EPS": 0.20,
    "ENT_COEF": 0., #0.01,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "NET_ARCH": (64, 32, 16), #(16, 16, 16),
    "LSTM_NET_ARCH": (24, 12),
    "ACTIVATION": 'tanh',
    "LSTM_ACTIVATION": 'tanh',
    "LOGISTIC_FUNCTION_TO_ACTOR": False,
    "ENV_NAME": None,
    "LR_SCHEDULE": 'cosine',
    'DEBUG': False,
    "NORMALIZE_REWARD_FOR_GAE_AND_TARGETS": False,
    "NORMALIZE_TARGETS": False,
    "NORMALIZE_ADVANTAGES": True,
    "ENV_TYPE": env_type,
    'NETWORK_TYPE_BATTERIES': 'actor_critic',
    'NETWORK_TYPE_REC': 'actor_critic',
    'USE_WEIGHT_DECAY': True,
    'NORMALIZE_NN_INPUTS': False
}

rng = jax.random.PRNGKey(42)

env = my_env_creator(params, battery_type, env_type=env_type)
env, train_state = make_train(config, env)

env_testing = my_env_creator(params_testing, battery_type, env_type=env_type)

val_rng = jax.random.PRNGKey(51)
val_num_iters = 8670*8

# out = train_jit(rng)

config = freeze(config)

norm? False
norm? False


In [10]:
config

FrozenDict({
    LR_BATTERIES: 5e-06,
    LR_REC: 5e-06,
    LR_BATTERIES_MIN: 5e-08,
    LR_REC_MIN: 5e-08,
    NUM_ENVS: 4,
    NUM_STEPS: 8192,
    TOTAL_TIMESTEPS: 385440,
    UPDATE_EPOCHS: 10,
    NUM_MINIBATCHES: 32,
    NORMALIZE_ENV: False,
    GAMMA: 0.99,
    GAE_LAMBDA: 0.98,
    CLIP_EPS: 0.2,
    ENT_COEF: 0.0,
    VF_COEF: 0.5,
    MAX_GRAD_NORM: 0.5,
    NET_ARCH: (64, 32, 16),
    LSTM_NET_ARCH: (24, 12),
    ACTIVATION: 'tanh',
    LSTM_ACTIVATION: 'tanh',
    LOGISTIC_FUNCTION_TO_ACTOR: False,
    ENV_NAME: None,
    LR_SCHEDULE: 'cosine',
    DEBUG: False,
    NORMALIZE_REWARD_FOR_GAE_AND_TARGETS: False,
    NORMALIZE_TARGETS: False,
    NORMALIZE_ADVANTAGES: True,
    ENV_TYPE: 'new_clip',
    NETWORK_TYPE_BATTERIES: 'actor_critic',
    NETWORK_TYPE_REC: 'actor_critic',
    USE_WEIGHT_DECAY: True,
    NORMALIZE_NN_INPUTS: False,
    NUM_UPDATES: 11,
    MINIBATCH_SIZE: 1024,
    BATTERY_ACTION_SPACE_SIZE: 1,
    BATTERY_OBSERVATION_SPACE_SIZE: 11,
    BATTERY_NUM_S

In [11]:
env.battery_agents

['battery_agent_0', 'battery_agent_1', 'battery_agent_2']

In [12]:
env.valorization_incentive_coeff

8e-06

In [13]:
networks_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)

In [14]:
optimizer_batteries

StackedOptimizer(
  model=StackedActorCritic(
    act_layers=[Linear(
      bias=Param(
        value=Array(shape=(3, 64), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x7fe4943efec0>,
      dot_general=<function dot_general at 0x7fe526177100>,
      dtype=None,
      in_features=11,
      kernel=Param(
        value=Array(shape=(3, 11, 64), dtype=float32)
      ),
      kernel_init=<function orthogonal.<locals>.init at 0x7fe4943efe20>,
      out_features=64,
      param_dtype=<class 'jax.numpy.float32'>,
      precision=None,
      use_bias=True
    ), <PjitFunction of <function tanh at 0x7fe525e0dd00>>, Linear(
      bias=Param(
        value=Array(shape=(3, 32), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x7fe4941a65c0>,
      dot_general=<function dot_general at 0x7fe526177100>,
      dtype=None,
      in_features=64,
      kernel=Param(
        value=Array(shape=(3, 64, 32), dtype=float32)
      ),
      kernel_init=<

In [15]:
networks_batteries

StackedActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(3, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7fe4943efec0>,
    dot_general=<function dot_general at 0x7fe526177100>,
    dtype=None,
    in_features=11,
    kernel=Param(
      value=Array(shape=(3, 11, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7fe4943efe20>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x7fe525e0dd00>>, Linear(
    bias=Param(
      value=Array(shape=(3, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7fe4941a65c0>,
    dot_general=<function dot_general at 0x7fe526177100>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(3, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7fe4941a6ca0>,
    out_features=32,
    par

In [16]:
network_rec

RECActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(64,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7fe49401e480>,
    dot_general=<function dot_general at 0x7fe526177100>,
    dtype=None,
    in_features=13,
    kernel=Param(
      value=Array(shape=(13, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7fe49401e520>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x7fe525e0dd00>>, Linear(
    bias=Param(
      value=Array(shape=(32,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7fe494096c00>,
    dot_general=<function dot_general at 0x7fe526177100>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7fe494096de0>,
    out_features=32,
    param_dtype=<clas

In [17]:
import time
rng = jax.random.PRNGKey(42)
t0 = time.time()

# with jax.checking_leaks():
out = train_wrapper(env, config, train_state, rng, validate=True, freq_val=10, val_env=env_testing, val_rng=val_rng, val_num_iters=val_num_iters, params=params, path_saving='/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER/trained_agents/')

train_state = out['runner_state'].train_state

print(f"time: {time.time() - t0:.2f} s")

Running for 11 iterations:   0%|          | 0/11 [00:00<?, ?it/s]

Running for 69,360 iterations:   0%|          | 0/69360 [00:00<?, ?it/s]

curr_iter: 0
	r_tot: -6967.6064453125
	r_trad: -3634.84033203125
	r_deg: -3620.55615234375
	r_clip: -79.29743957519531
	r_glob: 367.087158203125
	r_rec: 3014248.0
	mean soc: 0.5902138948440552
	std actions: [0.03721919 0.086419   0.02779719]
	self consumption: 3014248.0



Running for 69,360 iterations:   0%|          | 0/69360 [00:00<?, ?it/s]

curr_iter: 10
	r_tot: -6071.8671875
	r_trad: -3582.717529296875
	r_deg: -2822.734375
	r_clip: -31.865108489990234
	r_glob: 365.4500732421875
	r_rec: 3000507.25
	mean soc: 0.27927079796791077
	std actions: [0.04526298 0.04947678 0.02903032]
	self consumption: 3000507.25

Saving...
time: 355.23 s


In [18]:
# jax.tree.map(lambda x: x.shape, out['metrics'])

In [19]:
# jax.tree.map(lambda x: jnp.isnan(x).any(), out['metrics'])

In [20]:
# jax.tree.map(lambda x: jnp.isnan(x).any(axis=jnp.arange(1, x.ndim)), out['metrics'])

In [21]:
# out['metrics']['actions']['REC_agent'][0]

In [22]:
# out['metrics']['actions']['REC_agent'][1]

In [23]:
# np.isnan(out['metrics']['actions']['REC_agent'][1]).any()

In [24]:
# jax.tree.map(lambda x: np.nonzero(np.isnan(x)), out['metrics'])

In [25]:
# out['metrics']['actions']['REC_agent']

In [26]:
# out['metrics']['pure_reward']['r_clipping'][3]

In [27]:
network_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)
network_batteries

StackedActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(3, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7fe4943efec0>,
    dot_general=<function dot_general at 0x7fe526177100>,
    dtype=None,
    in_features=11,
    kernel=Param(
      value=Array(shape=(3, 11, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7fe4943efe20>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x7fe525e0dd00>>, Linear(
    bias=Param(
      value=Array(shape=(3, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7fe4941a65c0>,
    dot_general=<function dot_general at 0x7fe526177100>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(3, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7fe4941a6ca0>,
    out_features=32,
    par

## State saving

In [28]:
from algorithms.utils import save_state_multiagent
save_state_multiagent(network_batteries, network_rec, config, params, train_info=out['metrics'], val_info=out.get('val_info'), env_type=env_type, additional_info='only_trad_'+'total_timesteps'+str(total_timesteps)+'_init_full_'+str('full' in pack_options))

TypeError: save_state_multiagent() got an unexpected keyword argument 'env_type'

In [29]:
out

{'runner_state': RunnerState(train_state=TrainState(graph_def=NodeDef(
   type=tuple,
   index=-1,
   attributes=(0, 1, 2, 3),
   subgraphs={
     0: NodeDef(
       type=StackedActorCritic,
       index=0,
       attributes=('act_layers', 'cri_layers', 'log_std', 'normalize', 'num_networks'),
       subgraphs={
         'act_layers': NodeDef(
           type=list,
           index=-1,
           attributes=(0, 1, 2, 3, 4, 5, 6),
           subgraphs={
             0: NodeDef(
               type=Linear,
               index=1,
               attributes=('bias', 'bias_init', 'dot_general', 'dtype', 'in_features', 'kernel', 'kernel_init', 'out_features', 'param_dtype', 'precision', 'use_bias'),
               subgraphs={
                 'dtype': NodeDef(
                   type=NoneType,
                   index=-1,
                   attributes=(),
                   subgraphs={},
                   static_fields={},
                   leaves={},
                   metadata=None,
    